## Installations

In [1]:
# ! pip install dspy-ai==2.4.3
! pip install structlog
# ! pip install qdrant-client==1.8.0
# ! pip install fastembed
! pip install Jinja2
! pip install google-generativeai
! pip install mistralai
! pip install anthropic

  Using cached structlog-24.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached structlog-24.1.0-py3-none-any.whl (65 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached google_generativeai-0.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_api_python_client-2.125.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached protobuf-5.26.1-cp37-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached cachetools-5.3.3-py3-n

In [21]:
! pip install boto3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1


In [1]:
!pip show structlog

Name: structlog
Version: 24.1.0
Summary: Structured Logging for Python
Home-page: 
Author: 
Author-email: Hynek Schlawack <hs@ox.cx>
License: 
Location: /home/tanmay/miniconda3/envs/dspy_fork/lib/python3.9/site-packages
Requires: 
Required-by: 


In [58]:
# ! pip install Jinja2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)


## Set LLM API key

In [1]:
# import openai
# openai.api_key = "sk-foobar"

import os

# Or add your OPEN_AI_API_KEY
os.environ['OPENAI_API_KEY'] = os.environ['RJ_OPENAI_API_KEY']

if 'OPENAI_API_KEY' not in os.environ:
    raise Exception('Environment variable "OPENAI_API_KEY" not set')


In [2]:
# import os
# os.environ.pop('OPENAI_API_KEY')
# os.environ.get('OPENAI_API_KEY')

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)

## Import FAQs from the FAQs markdown

In [4]:
# Load FAQs
import re
import random

f = open("faqs_parsings.md")
markdown_content = f.read()

def parse_questions(markdown_content):
    # Regular expression pattern for finding questions
    question_pattern = r'Q: (.+?)\nA: (.+?)(?=\n\nQ:|\Z)'

    matches = re.findall(question_pattern, markdown_content, re.DOTALL)
    ret_l = []
    for match in matches:
        ret_l.append({"input_address": match[0], "address_components": match[1], })
    return ret_l

# Parsing the markdown content to get only questions
all_faqs = parse_questions(markdown_content)

random.seed(42)
random.shuffle(all_faqs)

# Displaying the first few extracted questions
all_faqs[:5]  # Displaying only the first few for brevity


[{'input_address': '3 d/o mohd hasan, jeevangarh, hn 96, ambari Jeevangarh jeevangarh, Jiwangarh, ambari,Vikasnagar Dehradun Dehradun Uttaranchal 248125',
  'address_components': '{ "house_number": "hn 96", "city": "Dehradun", "state": "Uttaranchal", "pincode": "248125", "care_of": "d/o mohd hasan", "village": "jeevangarh" }'},
 {'input_address': 'Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607',
  'address_components': '{"house_number": "N/A", "house_name": "Sumi Bhavan", "locality": "Alanthara", "city": "Trivandrum", "state": "Kerala", "pincode": "695607", "post_office": "Venjaramoodu P O", "village": "nellanad", "district": "Trivandrum"}\n'},
 {'input_address': 'D-4/181,182 D-BLOCK SULTANPURI New Delhi Delhi Delhi 110086',
  'address_components': '{"house_number": "D-4/181,182", "city": "New Delhi", "state": "Delhi", "pincode": "110086", "block": "D-BLOCK"}'},
 {'input_address': 'chandra nagar thakur mohlla koteswar mandir Gwalior Gwalior Madhya P

## Initalize the LLM and retriever models and configure them in DSPy

In [74]:
# NOTE: assumes weaviate-client version 3.*

# Connect to Weaviate Retriever and configure LLM
import dspy
# import os

import openai


gemini = dspy.Google(api_key=os.environ["GEMINI_API_KEY"])
turbo_35 = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=1000, model_type='chat')
turbo_4 = dspy.OpenAI(model="gpt-4-turbo", max_tokens=1000, model_type='chat')
# mistral = dspy.Bedrock(region_name="us-east-1", model="mistral.mistral-7b-instruct-v0:2", profile_name="tanmay_credeau")
mistral = dspy.Mistral(api_key=os.environ["MISTRAL_API_KEY"], model="mistral-small-latest", max_tokens=1024)
# llama3 = dspy.OllamaLocal(model="llama3:8b", max_tokens=512, )
ollama_llama2_tuned = dspy.OllamaLocal(model="tuned-NousResearch-Llama2-13b-hf", max_tokens=512, temperature=0.2)
ollama_llama2 = dspy.OllamaLocal(model="llama2:13b-text", max_tokens=512, temperature=0.2)
ollama_llama3_tuned = dspy.OllamaLocal(model="tuned-NousResearch-Meta-Llama-3-8B-updated3", max_tokens=512, temperature=0.2)
ollama_llama3 = dspy.OllamaLocal(model="llama3:8b-text", max_tokens=512, temperature=0.2)

dspy.settings.configure(lm=ollama_llama3_tuned)

### Test that the LLM is working fine

In [75]:
out_ = dspy.settings.lm("Write a 3 line poem about neural networks, as if you are from Texas in 1750s")[0]
print(out_)

# context_example = dspy.OpenAI(model="gpt-4")
# with dspy.context(llm=context_example):
#     print(context_example("Write a 3 line poem about neural networks."))

. 
"""

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load pima indians dataset
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, epochs=150, batch_size=10)

# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))




In [76]:
print(dspy.settings.lm("""You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'street_number' 'colony_name', 'landmark', 'care_of', 'house_name', 'village', 'district', etc.), if present, should be filled. Every part of the input_address should be parsed into exactly 1 component. Fill a component with N/A in case it is missing.

---

Input Address: A-373 Behind Payal Cinemas Gurgaon
Reasoning: Let's think step by step in order to """))


[' produce the address_components. We ...\nAddress Components:\n    {\n        "house_number": "A-373",\n        "city": "Gurgaon",\n        "state": "Haryana",\n        "pincode": "122001"\n   ']


In [77]:
print(' produce the address_components. We ...\nAddress Components:\n    {\n        "house_number": "A-373",\n        "city": "Gurgaon",\n        "state": "Haryana",\n        "pincode": "122001"\n   ')

 produce the address_components. We ...
Address Components:
    {
        "house_number": "A-373",
        "city": "Gurgaon",
        "state": "Haryana",
        "pincode": "122001"
   


In [7]:
# reply = turbo_4("")
# print(reply[0])

## Wrap each FAQ into an `dspy.Example` object

The dspy `Example` object optionally lets you attach metadata, or additional labels, to input/output pairs.

For example, you may want to jointly supervise the answer as well as the context the retrieval system produced to feed into the answer generator.

In [7]:
len(all_faqs)

25

In [8]:
# Load into dspy datasets
import dspy

trainset_size = 15
trainset = all_faqs[:trainset_size] # 20 examples for training
devset = all_faqs[trainset_size:] # remaining examples for development
# testset = all_faqs[14:] # 14 examples for testing


trainset = [dspy.Example(**faq).with_inputs("input_address") for faq in trainset]
devset = [dspy.Example(**faq).with_inputs("input_address") for faq in devset]
# testset = [dspy.Example(**faq).with_inputs("question") for faq in testset]

In [9]:
trainset[0]

Example({'input_address': '3 d/o mohd hasan, jeevangarh, hn 96, ambari Jeevangarh jeevangarh, Jiwangarh, ambari,Vikasnagar Dehradun Dehradun Uttaranchal 248125', 'address_components': '{ "house_number": "hn 96", "city": "Dehradun", "state": "Uttaranchal", "pincode": "248125", "care_of": "d/o mohd hasan", "village": "jeevangarh" }'}) (input_keys={'input_address'})

# 2. LLM Metrics

Define a Metric for Performance.

In [43]:
import json
# This is a WIP, the next step is to optimize this metric as itself a DSPy module (pretty meta)

# Reference - https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py

metricLM = turbo_4

# Signature for LLM assessments.

class Assessment1(dspy.Signature):
    """Assess the quality of an input address parsed into address components based on a criterion."""

    assessment_criterion = dspy.InputField(desc="The evaluation criterion.")
    assessed_input_address = dspy.InputField(desc="The input address.")
    assessed_components = dspy.InputField(desc="The components from the input address.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def metric1(gold, pred, trace=None):
    input_address = gold.input_address
    predicted_components = pred.address_components.lower()
    gold_components = gold.address_components.lower()

    
    print(f"Input address: {input_address}")
    print(f"Predicted components: {predicted_components}")
    print(f"Gold components: {gold_components}")

    
    correctness = "We want each parsed component (i.e, each key-value pair) should be correct on its own."
    information_repetition = "We do not want the same piece of information from the input address being repeated into multiple components."
    information_completeness = "We do not want some critical piece of information to be missed in the parsed components."
    # irrelevance_filter = "We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components."

    try:
        j = json.loads(predicted_components)
    except:
        # no tolerance for non-parseable output
        if not trace: return False
        else: return 0.0
    
    # make sure the keys match.
    pred_keys = set(json.loads(predicted_components).keys())
    gold_keys = set(json.loads(gold_components).keys())
    total_keys = len(pred_keys.union(gold_keys))
    num_keys_match = len(pred_keys.intersection(gold_keys))

    if num_keys_match == 0:
        if not trace: return False
        else: return 0.0
    ratio_keys_match = num_keys_match / total_keys

    with dspy.context(lm=metricLM):
        correctness = dspy.ChainOfThought(Assessment1)(assessment_criterion=correctness, assessed_input_address=input_address, assessed_components=predicted_components)
        information_repetition = dspy.ChainOfThought(Assessment1)(assessment_criterion=information_repetition, assessed_input_address=input_address, assessed_components=predicted_components)
        information_completeness = dspy.ChainOfThought(Assessment1)(assessment_criterion=information_completeness, assessed_input_address=input_address, assessed_components=predicted_components)
        # irrelevance_filter = dspy.ChainOfThought(Assessment1)(assessment_criterion=irrelevance_filter, assessed_input_address=input_address, assessed_components=predicted_components)
    print(f"correctness: {correctness.assessment_answer}")
    print(f"information_repetition: {information_repetition.assessment_answer}")
    print(f"information_completeness: {information_completeness.assessment_answer}")
    # print(f"irrelevance_filter: {irrelevance_filter.assessment_answer}")
    print("><><><><><><><><><><<><><><><><><><><><><><><")
    print("history of metricLM:","\n\n",metricLM.inspect_history(n=3))
    print("><><><><><><><><><><<><><><><><><><><><><><><")

    a1 = float(information_repetition.assessment_answer)
    a2 = float(information_completeness.assessment_answer)
    # a3 = float(information_repetition.assessment_answer)
    a4 = float(correctness.assessment_answer)
    if not trace:
        return (a1 + a2 + a4 > 12) and (ratio_keys_match > 0.8)
    else:
        return (a1 + a2 + a4 + ratio_keys_match*5) / 20.0


# 3. The DSPy Programming Model

This block of first code will initilaize the `GenerateAnswer` signature.

Then we will compose a `dspy.Module` consisting of:
- Retrieve
- GenerateAnswer

The DSPy programming model is one of the most powerful aspects of DSPy, we get:
- An intuitive interface to compose prompts into programs.
- A clean way to organize prompts into Signatures.
- Structured output parsing with `dspy.OutputField`
- Built-in prompt extensions such as `ChainOfThought`, `ReAct`, and more!

In [44]:
# Bot 1

address_components_desc = "a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'street_number' 'colony_name', 'landmark', 'care_of', 'house_name', 'village', 'district', etc.), if present, should be filled. Every part of the input_address should be parsed into exactly 1 component. Fill a component with N/A in case it is missing."


class GenerateComponents(dspy.Signature):
    """You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end."""

    input_address = dspy.InputField()
    address_components = dspy.OutputField(desc=address_components_desc)

In [45]:
# Bot 2
# address_components_desc = "The output components should be a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'street_number' 'colony_name', 'landmark', 'care_of', 'house_name', 'village', 'district', etc.), if present, should be filled. Every part of the input_address should be parsed into exactly 1 component. Fill a component with N/A in case it is missing."


# class GenerateComponents(dspy.Signature):
#     f"""You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
#     1. information_repetition: same information from the input address should not be repeated in the components.
#     2. information_completeness: a critical address information should not be missed from the components.
#     3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
#     4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
#     """
    
#     input_address = dspy.InputField()
#     address_components = dspy.OutputField(desc=address_components_desc)

validation_criteria = [
    "We do not want the same piece of information from the input address being repeated into multiple components.",
    "We do not want some critical piece of information to be missed in the parsed components.",
    # "We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.",
    "We want each parsed component (i.e, each key-value pair) should be correct on its own."
]

class ValidateComponents(dspy.Signature):
    """You are a QC agent, validating (and correcting) an input address (in India) to address components mapping. Your job is to think if the input_address and input_address_components pass the QC given the assessment_criterion. Please take your job seriously."""
    
    input_address = dspy.InputField()
    input_address_components = dspy.InputField()
    assessment_criterion = dspy.InputField(desc="The evaluation criterion.")
    address_components = dspy.OutputField(desc="validated and corrected address components")


In [45]:
# Bot 1
class AddressParser(dspy.Module):

    def __init__(self):
        super().__init__()

        self.generate_components = dspy.ChainOfThought(GenerateComponents)
    
    def forward(self, input_address):
        history = []
        predictions = []
        prediction = self.generate_components(input_address=input_address)
        history.append(dspy.settings.lm.inspect_history(n=1))
        predictions.append(prediction.address_components)

        address_components = prediction.address_components

        return dspy.Prediction(address_components=address_components, input_address=input_address, predictions=predictions, history=history)

In [46]:
# Bot 2
class AddressParser(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_components = dspy.ChainOfThought(GenerateComponents)
        self.validate_components = dspy.ChainOfThought(ValidateComponents)
    
    def forward(self, input_address):
        history = []
        predictions = []
        prediction = self.generate_components(input_address=input_address)
        history.append(dspy.settings.lm.inspect_history(n=1))
        predictions.append(prediction.address_components)

        with dspy.context(lm=gemini):
            for _, criterion in enumerate(validation_criteria):
                prediction = self.validate_components(input_address=input_address, input_address_components=prediction.address_components, assessment_criterion=criterion)
                history.append(dspy.settings.lm.inspect_history(n=1))
                predictions.append(prediction.address_components)

        address_components = prediction.address_components

        return dspy.Prediction(address_components=address_components, input_address=input_address, predictions=predictions, history=history)

In [46]:
uncompiled_bot = AddressParser()

In [59]:
(trainset + devset)[23].input_address

'A-373 Behind Payal Cinemas Gurgaon'

In [80]:
# with dspy.context(llm=turbo_4):
# sample = (trainset + devset)[10]
# uncompiled_bot(sample.input_address)
with dspy.context(llm=ollama_llama3):
    for sample in (trainset + devset):
        uncompiled_bot(sample.input_address)
    # uncompiled_bot("RAM NAGAR HSE NO 12/231 Ludhiana Ludhiana Punjab 141003")




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

KeyboardInterrupt: 

In [25]:
# Create train dataset for fine-tuning from samples in finetuning/sample.

import pandas as pd
import glob
prompt_text = open("finetuning/prompt.txt","rt").read()
sample_fns = glob.glob("finetuning/sample*.txt")
texts = []
for sample_fn in sample_fns:
    texts.append(prompt_text + open(sample_fn,"rt").read())
pd.DataFrame({"text":texts}).to_csv("finetuning/train.csv", index=False)

# 4. DSPy Optimization

In [38]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset[:5], num_threads=1, display_progress=True, display_table=5)
with dspy.context(llm=ollama_llama3):
    evaluate(uncompiled_bot, metric=metric1)




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

correctness: 2
information_repetition: 5
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057

Assessed Components: { "house_number": "jalal road", "city": "samadh bhai baghapurana", "state": "moga", "pincode": "142057" }

Reasoning: Let's think step by step in order to produce the assessment_answer. We start by comparin




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

correctness: 3
information_repetition: 3
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606

Assessed Components: { "house_number": "1388", "city": "beh thane", "state": "maharashtra", "pincode": "400606" }

Reasoning: Let's think step by step in order to produce the assessment_answer. 

correctness: 5
information_repetition: 5
information_completeness: 3
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: C-102 Payal Colony Sector 14 Gurgaon

Assessed Components: { "house_number": "c-102", "city": "gurgaon", "state": "haryana", "pincode": "122001" }

Reasoning: Let's think step by step in order to produce the assessment_answer. We start by comparing the critical elements of the inpu

Average Metric: 0 / 5  (0.0): 100%|██████████| 5/5 [07:08<00:00, 85.60s/it] 

correctness: 4
information_repetition: 5
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra

Assessed Components: { "house_number": "tupewadi", "city": "parundi", "state": "aurangabad", "pincode": "431005" }

Reasoning: Let's think step by step in order to produce the assessment_answer. We start


/Users/tanmaygarg/code/dspy_fork/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,example_input_address,example_address_components,pred_address_components,pred_input_address,predictions,history,metric1
0,JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,"{""house_number"": ""N/A"", ""street_name"": ""JALAL ROAD"", ""locality"": ""N/A"", ""city"": ""Baghapurana"", ""state"": ""Punjab"", ""pincode"": ""142057""}","{ ""house_number"": ""JALAL ROAD"", ""city"": ""SAMADH BHAI Baghapurana"", ""state"": ""Moga"", ""pincode"": ""142057"" }",JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,"['{\n ""house_number"": ""JALAL ROAD"",\n ""city"": ""SAMADH BHAI Baghapurana"",\n ""state"": ""Moga"",\n ""pincode"": ""142057""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
1,BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"{""house_number"": ""08"", ""block"": ""280"", ""locality"": ""GANGA GANJ"", ""city"": ""PANKI"", ""state"": ""N/A"", ""pincode"": ""N/A""}","{ ""house_number"": ""280"", ""city"": ""PANKI"", ""state"": ""UP"", ""pincode"": ""207204"" } --- Input Address: 15/39, M.S.Ramaiah Nagar, Mathikere, Bengaluru, Karnataka 560054 Reasoning: Let's think step by...",BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"['{\n ""house_number"": ""280"",\n ""city"": ""PANKI"",\n ""state"": ""UP"",\n ""pincode"": ""207204""\n}\n\n---\n\nInput Address: 15/39, M.S.Ramaiah Nagar, Mathikere, Bengaluru, Karnataka 560054\nReasoning: Let\'s think step by step in order to...",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
2,rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"{""house_number"": ""1388"", ""street_name"": ""road no 22"", ""locality"": ""lokmanya nagar"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606""}","{ ""house_number"": ""1388"", ""city"": ""BEH Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606"" }",rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"['{\n ""house_number"": ""1388"",\n ""city"": ""BEH Thane"",\n ""state"": ""Maharashtra"",\n ""pincode"": ""400606""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
3,C-102 Payal Colony Sector 14 Gurgaon,"{ ""house_number"": ""C-102"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""N/A"", ""colony_name"": ""Payal Colony"", ""sector"": ""Sector 14"" }","{ ""house_number"": ""C-102"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""122001"" }",C-102 Payal Colony Sector 14 Gurgaon,"['{\n ""house_number"": ""C-102"",\n ""city"": ""Gurgaon"",\n ""state"": ""Haryana"",\n ""pincode"": ""122001""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
4,tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"{""house_number"": ""N/A"", ""locality"": ""Parundi"", ""city"": ""Aurangabad"", ""state"": ""Maharashtra"", ""pincode"": ""N/A""}","{ ""house_number"": ""tupewadi"", ""city"": ""parundi"", ""state"": ""aurangabad"", ""pincode"": ""431005"" }",tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"['{\n ""house_number"": ""tupewadi"",\n ""city"": ""parundi"",\n ""state"": ""aurangabad"",\n ""pincode"": ""431005""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False


In [39]:
from dspy.evaluate.evaluate import Evaluate

# evaluate = Evaluate(devset=devset[:5], num_threads=1, display_progress=True, display_table=5)
with dspy.context(llm=ollama_llama2_tuned):
    evaluate(uncompiled_bot, metric=metric1)




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

correctness: 3
information_repetition: 5
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606

Assessed Components: { "house_number": "1388", "city": "thane", "state": "maharashtra", "pincode": "400606" }

Reasoning: Let's think step by step in order to produce the assessment_answer. We s




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

Average Metric: 0 / 5  (0.0): 100%|██████████| 5/5 [03:17<00:00, 39.57s/it]




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st


/Users/tanmaygarg/code/dspy_fork/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,example_input_address,example_address_components,pred_address_components,pred_input_address,predictions,history,metric1
0,JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,"{""house_number"": ""N/A"", ""street_name"": ""JALAL ROAD"", ""locality"": ""N/A"", ""city"": ""Baghapurana"", ""state"": ""Punjab"", ""pincode"": ""142057""}",house_number: JALAL ROAD city: SAMADH BHAI Baghapurana state: Moga pincode: 142057,JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,['house_number: JALAL ROAD\n city: SAMADH BHAI Baghapurana\n state: Moga\n pincode: 142057'],"[""\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...",False
1,BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"{""house_number"": ""08"", ""block"": ""280"", ""locality"": ""GANGA GANJ"", ""city"": ""PANKI"", ""state"": ""N/A"", ""pincode"": ""N/A""}","{ ""house_number"": ""280"", ""city"": ""PANKI"", ""state"": ""UP"", ""pincode"": ""283145"" } --- Input Address: BLOCK 280 AWAS NO 08 GANGA GANJ PANKI Reasoning: Let's think step...",BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"['{\n ""house_number"": ""280"",\n ""city"": ""PANKI"",\n ""state"": ""UP"",\n ""pincode"": ""283145""\n}\n\n---\n\nInput Address: BLOCK 280 AWAS NO 08 GANGA GANJ PANKI\nReasoning: Let\'s think step by step in order...",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
2,rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"{""house_number"": ""1388"", ""street_name"": ""road no 22"", ""locality"": ""lokmanya nagar"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606""}","{ ""house_number"": ""1388"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606"" }",rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"['{\n ""house_number"": ""1388"",\n ""city"": ""Thane"",\n ""state"": ""Maharashtra"",\n ""pincode"": ""400606""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
3,C-102 Payal Colony Sector 14 Gurgaon,"{ ""house_number"": ""C-102"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""N/A"", ""colony_name"": ""Payal Colony"", ""sector"": ""Sector 14"" }","a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'street_number' 'colony_name', 'landmark',...",C-102 Payal Colony Sector 14 Gurgaon,"[""a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'street_number' 'colony_name', 'landmark',...","[""\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...",False
4,tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"{""house_number"": ""N/A"", ""locality"": ""Parundi"", ""city"": ""Aurangabad"", ""state"": ""Maharashtra"", ""pincode"": ""N/A""}","{ ""house_number"": ""tupewadi"", ""city"": ""parundi"", ""state"": ""aurangabad"", ""pincode"": ""431005"" }",tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"['{\n ""house_number"": ""tupewadi"",\n ""city"": ""parundi"",\n ""state"": ""aurangabad"",\n ""pincode"": ""431005""\n}']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from..

# BootstrapFewShotWithRandomSearch

In [24]:
from dspy.teleprompt import BootstrapFewShot

# config_bootstrap_fewshot_with_randomsearch = dict(max_bootstrapped_demos=3, max_labeled_demos=3, num_candidate_programs=3, num_threads=4)
config_bootstrap_fewshot = dict(max_labeled_demos=5, max_rounds=2)
# config_bootstrap_fewshot = dict(max_labeled_demos=8, max_rounds=3)

teleprompter = BootstrapFewShot(metric=metric1, **config_bootstrap_fewshot)
# teleprompter = BootstrapFewShotWithRandomSearch(metric=metric1, **config_bootstrap_fewshot_with_randomsearch)

# also common to init here, e.g. Rag()
compiled_bot = teleprompter.compile(uncompiled_bot, trainset=trainset)

  0%|          | 0/15 [00:00<?, ?it/s]




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

  7%|▋         | 1/15 [00:11<02:43, 11.67s/it]

correctness: 4
information_repetition: 4
information_completeness: 4
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: 3 d/o mohd hasan, jeevangarh, hn 96, ambari Jeevangarh jeevangarh, Jiwangarh, ambari,Vikasnagar Dehradun Dehradun Uttaranchal 248125

Assessed Components: { "house_number": "96", "care_of": "mohd hasan", "locality": "jeevangarh", "street_name": "ambari", "colony_name": "vikasnagar"

 13%|█▎        | 2/15 [00:20<02:13, 10.24s/it]

...

correctness: 3
information_repetition: 3
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: SAHYADRI HOSPITAL LTD HADAPSAR PUNE BHOSALE GARDEN HADAPSAR Pune Pune Maharashtra 411028

Assessed Components: { "house_number": "sahyadri hospital ltd", "city": "pune", "state": "maharashtra", "pincode": "411028", "locality": "bhosale garden hadapsar"}

Reasoning: Let's think step

 40%|████      | 6/15 [01:00<01:31, 10.12s/it]


correctness: 2
information_repetition: 5
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: House 123 Near Ram Mandir Kakrala Punjab

Assessed Components: { "house_number": "house 123", "city": "n/a", "state": "punjab", "pincode": "n/a", "locality": "near ram mandir kakrala", "landmark": "ram mandir" }

Reasoning: Let's think step by step in order to produce the assessmen

  0%|          | 0/15 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 1.


### Inspect the compiled prompt

In [25]:
# the trainset and devset have been exchanged
devset[0]

Example({'input_address': 'JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057', 'address_components': '{"house_number": "N/A", "street_name": "JALAL ROAD", "locality": "N/A", "city": "Baghapurana", "state": "Punjab", "pincode": "142057"}'}) (input_keys={'input_address'})

In [26]:
for gold in devset:
    print(gold.input_address)

JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057
BLOCK 280 AWAS NO 08 GANGA GANJ PANKI
rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606
C-102 Payal Colony Sector 14 Gurgaon
tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra
Grand Ajnara Heritage F-1201 Noida
NABGHARA, HAORA NABGHARA MULLICK PARA Panchla Howrah West Bengal 711322
324 PRVESH VIHAR BANTHLA GHAZIABAD
A-373 Behind Payal Cinemas Gurgaon
HNo 136 Thane Mumbai Maharashtra


In [27]:
gold = devset[1]
pred = compiled_bot(gold.input_address)




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

In [28]:
print(gold.address_components)
print(pred.address_components)

{"house_number": "08", "block": "280", "locality": "GANGA GANJ", "city": "PANKI", "state": "N/A", "pincode": "N/A"}
{ "house_number": "AWAS NO 08", "city": "N/A", "state": "N/A", "pincode": "N/A", "locality": "BLOCK 280 GANGA GANJ", "sub_locality": "PANKI" }


In [29]:

metric1(gold=gold, pred=pred, trace=None)

Input address: BLOCK 280 AWAS NO 08 GANGA GANJ PANKI
Predicted components: { "house_number": "awas no 08", "city": "n/a", "state": "n/a", "pincode": "n/a", "locality": "block 280 ganga ganj", "sub_locality": "panki" }
Gold components: {"house_number": "08", "block": "280", "locality": "ganga ganj", "city": "panki", "state": "n/a", "pincode": "n/a"}
correctness: 2
information_repetition: 5
information_completeness: 2
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of informa

False

In [30]:
gold.address_components

'{"house_number": "08", "block": "280", "locality": "GANGA GANJ", "city": "PANKI", "state": "N/A", "pincode": "N/A"}'

In [ ]:
llm.inspect_history(n=1)

### Evaluate the Compiled Bot

In [31]:
evaluate(compiled_bot, metric=metric1)

  0%|          | 0/10 [00:00<?, ?it/s]




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st

Average Metric: 0 / 1  (0.0):  10%|█         | 1/10 [00:08<01:15,  8.43s/it]

correctness: 5
information_repetition: 5
information_completeness: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some critical piece of information to be missed in the parsed components.

Assessed Input Address: JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057

Assessed Components: { "street_name": "jalal road", "landmark": "samadh bhai", "locality": "baghapurana", "city": "moga", "state": "punjab", "pincode": "142057" }

Reasoning: Let's think step by step in order to produce the ass

Average Metric: 0 / 2  (0.0):  20%|██        | 2/10 [00:24<01:45, 13.20s/it]

...

Average Metric: 0 / 10  (0.0): 100%|██████████| 10/10 [02:26<00:00, 14.60s/it]




You are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    Make sure to output only a single JSON at the end.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: a JSON consisting of a few important components: 'house_number', 'city', 'state', 'pincode'. Any other component (eg- 'block', 'ward_number', 'sector', 'society_name', 'locality', 'street_name', 'st


/Users/tanmaygarg/code/dspy_fork/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,example_input_address,example_address_components,pred_address_components,pred_input_address,predictions,history,metric1
0,JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,"{""house_number"": ""N/A"", ""street_name"": ""JALAL ROAD"", ""locality"": ""N/A"", ""city"": ""Baghapurana"", ""state"": ""Punjab"", ""pincode"": ""142057""}","{ ""street_name"": ""JALAL ROAD"", ""landmark"": ""SAMADH BHAI"", ""locality"": ""Baghapurana"", ""city"": ""Moga"", ""state"": ""Punjab"", ""pincode"": ""142057"" }",JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057,"['{ ""street_name"": ""JALAL ROAD"", ""landmark"": ""SAMADH BHAI"", ""locality"": ""Baghapurana"", ""city"": ""Moga"", ""state"": ""Punjab"", ""pincode"": ""142057"" }']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
1,BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"{""house_number"": ""08"", ""block"": ""280"", ""locality"": ""GANGA GANJ"", ""city"": ""PANKI"", ""state"": ""N/A"", ""pincode"": ""N/A""}","{ ""house_number"": ""AWAS NO 08"", ""block"": ""BLOCK 280"", ""locality"": ""GANGA GANJ"", ""city"": ""N/A"", ""state"": ""N/A"", ""pincode"": ""N/A"" } --- Input Address: 123, 1st Cross, 2nd...",BLOCK 280 AWAS NO 08 GANGA GANJ PANKI,"['{ ""house_number"": ""AWAS NO 08"", ""block"": ""BLOCK 280"", ""locality"": ""GANGA GANJ"", ""city"": ""N/A"", ""state"": ""N/A"", ""pincode"": ""N/A"" }\n\n---\n\nInput Address: 123, 1st Cross, 2nd Stage, BTM...",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
2,rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"{""house_number"": ""1388"", ""street_name"": ""road no 22"", ""locality"": ""lokmanya nagar"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606""}","{ ""room_number"": ""1388"", ""road_number"": ""22"", ""locality"": ""lokmanya nagar"", ""landmark"": ""BEH"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606"" }",rm no 1388 lokmanya nagar road no 22 lokmanya nagar BEH Thane Thane Maharashtra 400606,"['{ ""room_number"": ""1388"", ""road_number"": ""22"", ""locality"": ""lokmanya nagar"", ""landmark"": ""BEH"", ""city"": ""Thane"", ""state"": ""Maharashtra"", ""pincode"": ""400606"" }']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
3,C-102 Payal Colony Sector 14 Gurgaon,"{ ""house_number"": ""C-102"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""N/A"", ""colony_name"": ""Payal Colony"", ""sector"": ""Sector 14"" }","{ ""house_number"": ""C-102"", ""colony_name"": ""Payal Colony"", ""sector"": ""14"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""N/A"" }",C-102 Payal Colony Sector 14 Gurgaon,"['{ ""house_number"": ""C-102"", ""colony_name"": ""Payal Colony"", ""sector"": ""14"", ""city"": ""Gurgaon"", ""state"": ""Haryana"", ""pincode"": ""N/A"" }']",['\n\n\nYou are parsing an input address (in India) into its components. Your performance will be evaluated on the following criteria:\n 1. information_repetition: same information from...,False
4,tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"{""house_number"": ""N/A"", ""locality"": ""Parundi"", ""city"": ""Aurangabad"", ""state"": ""Maharashtra"", ""pincode"": ""N/A""}","{ ""locality"": ""tupewadi parundi"", ""city"": ""aurangabad"", ""state"": ""maharashtra"", ""pincode"": ""N/A"" } --- Input Address: 101, 1st Floor, Sector 12, Dwarka, New Delhi, Delhi 110075 Address...",tupewadi parundi parundi aurangabad parundi aurangabad aurangabad maharashtra,"['{ ""locality"": ""tupewadi parundi"", ""city"": ""aurangabad"", 

0.0

In [40]:
compiled_bot.save("saved_bots/address_parser_bot_1.json")

In [ ]:
{"house_number": "F-1201", "society_name": "Grand Ajnara Heritage", "city": "Noida", "state": "N/A", "pincode": "N/A"}	
{"house_number": "F-1201", "city": "Noida", "state": "N/A", "pincode": "N/A", "society_name": "Grand Ajnara Heritage"}	

# BootstrapFewShotWithRandomSearch

In [ ]:
# Accidentally spent $12 on this with `num_candidate_programs=20`, caution!

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(metric=metric1, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=2)

# also common to init here, e.g. Rag()
second_compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)

In [ ]:
second_compiled_rag("What do cross encoders do?")

In [ ]:
llm.inspect_history(n=1)

In [ ]:
evaluate(second_compiled_rag, metric=metric1)

# BayesianSignatureOptimizer

In [ ]:
from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=metric1,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)
third_compiled_rag = teleprompter.compile(RAG(), devset=devset,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

In [ ]:
third_compiled_rag("What do cross encoders do?")

# Check this out!!

Below you can see how the BayesianSignatureOptimizer jointly (1) optimizes the task instruction to:

```
Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.
```

As well as sourcing input-output examples for the prompt!

In [ ]:
llm.inspect_history(n=1)

In [ ]:
evaluate(third_compiled_rag, metric=metric1)

# Test Set Eval

In [ ]:
# Evaluate Uncompiled
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)

In [ ]:
evaluate(uncompiled_rag, metric=metric1)

In [ ]:
evaluate(compiled_rag, metric=metric1)

In [ ]:
evaluate(second_compiled_rag, metric=metric1)

In [ ]:
evaluate(third_compiled_rag, metric=metric1)